# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import wget
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor
import requests

# Webdriver and Setting up the Link

In [2]:
path_of_chromedriver = "D:/Softwares/chromedriver_win32/chromedriver.exe"

service = Service(path_of_chromedriver)
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service , options=op)

link = "https://b2b.luminafashion.com/it/20-collezione?page=1"
driver.get(link)
time.sleep(10)


"""
Check if the page loads.
"""

In [3]:
account = driver.find_element(by = By.XPATH , value = "/html/body/main/header/div[1]/div[1]/div[1]/div[3]/ul/li[1]/div")
account.click()
time.sleep(5)

login = driver.find_element(by = By.XPATH , value = "/html/body/main/div[1]/div/div[4]/div/a[1]")
login.click()
time.sleep(5)

username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
username.clear()
username.send_keys("conidig@gmail.com")
username.send_keys(Keys.RETURN)
time.sleep(3)

password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))
password.clear()
password.send_keys("E06UCUD")
password.send_keys(Keys.RETURN)
time.sleep(10)



"""
Check, if the page has loaded successfully
"""

In [4]:
original_link = "https://b2b.luminafashion.com/it/20-collezione?page=1"
driver.get(original_link)
time.sleep(10)


"""
Check again
"""

## This part will scroll down all the way down.

In [5]:
num=0

In [10]:
scroll = True

while scroll==True:
    try:
        time.sleep(5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)
        loadmore = driver.find_element(by = By.ID,value="loadmoder")
        loadmore.click()
        time.sleep(5)
        num+=1
        print(num)
        
    except:
        scroll=False
        
        
"""
Check. Has scrolling completely finished. If NOT, run this cell again.
"""

## Scrolling has been done, now let's store the link of each product.

In [11]:
LINKS = []

In [12]:
html = driver.page_source
strainer = SoupStrainer('div', attrs={"class":"product-image image-load"})
soup = BeautifulSoup(html , 'html.parser' , parse_only=strainer)
divs = soup.find_all("div" , class_="product-image image-load")
print("Number of Products:", len(divs))

for i in divs:
    product_url = i.a['href']
    LINKS.append(product_url)
    
print("Number of Links", len(LINKS))

Number of Products: 394
Number of Links 394


# Scrapping Begins

In [187]:
TITLE = []
PRICE = []
SHORT_DESCRIPTION = []
DESCRIPTION2 = []
SIZES = []
COLOR = []
IMAGE1, IMAGE2, IMAGE3, IMAGE4 = [],[],[],[]
IMAGES = [IMAGE1, IMAGE2, IMAGE3, IMAGE4]




PRODUCT_NAME = []
SKU = []
COLOR_VARIANT = []
SIZE_VARIANT = []
AVAILABILITY = []

In [263]:
def looprun(p_url):
    try:
        driver.get(p_url)
    except:
        driver.get(p_url)
    time.sleep(7)

    # click on long description
    desc2 = driver.find_element(by = By.CSS_SELECTOR,value="input[id='chck1']")
    driver.execute_script("arguments[0].click();", desc2)
    time.sleep(3)

    # getting page source
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2 , 'html.parser')

    def text_with_newlines(elem):
        text = ''
        for e in elem.descendants:
            if isinstance(e, str):
                text += e
            elif e.name == 'br' or e.name == 'p':
                text += '\n'
        return text

    # title & product name
    title = soup2.find("h1" , class_="h1 product-title").text 
    # SKU
    try:
        sku = title.split(". ")[1] # sku
    except:
        sku = title.split(".")[1]
    # PRICE
    price = soup2.find("span", itemprop="price").text.replace(",",".").replace("\xa0€","")

    # Short Description
    short_des = soup2.find('div',itemprop="description")
    short_description = text_with_newlines(short_des).strip()
    
    # Long Description
    long_des = soup2.find('div', class_="tab-content product-description")
    description2 =  text_with_newlines(long_des).strip()



    # Variants: SIZE and COLORS
    variants = soup2.find_all('div',class_="clearfix product-variants-item")

    aa = variants[0]
    bb = variants[1]
    ul_size = aa.find('ul', id="group_1")
    ul_color = bb.find('ul', id="group_2")

    size_list = ul_size.find_all('li')
    color_list = ul_color.find_all('li')
    size=""
    color = ""
    for i in size_list:
        size += i.text.strip() + ","
    size=size[:-1]
    for i in color_list:
        color += i.text.strip()+"\n"
    color=color[:-1]



    colors_list = color.split("\n")
    size_list = size.split(",")

    ###################################################################################
    def size_n_color():
        size_clicks = driver.find_element(by=By.ID, value='group_1')
        color_clicks =driver.find_element(by=By.ID, value='group_2')
        global sc
        global cc

        sc = size_clicks.find_elements(by=By.CSS_SELECTOR, value="input[name='group[1]']")
        cc = color_clicks.find_elements(by=By.CSS_SELECTOR, value="input[name='group[2]']")
    ###################################################################################

    size_n_color()











    # Clicking on each color, and getting data.
    for COL in range(len(cc)):
        

        

        driver.execute_script("arguments[0].click();", cc[COL])
        time.sleep(5)
        
        current_URL = driver.current_url
        driver.get(current_URL)
        time.sleep(3)
        size_n_color()

        link_list = []
        ls = driver.find_elements(by=By.CLASS_NAME, value= "thumb-container")

        length = len(ls)
        if length>5:
            length=5

        for i in range(1, length):
            lnk = ls[i].find_element(by= By.TAG_NAME, value= "img").get_attribute('src')
            link_list.append(lnk)    








        
#####################################################################################################
        # clicking on Sizes and getting data about its availability.
        for j in range(len(sc)):
            driver.execute_script("arguments[0].click();", sc[j])
            time.sleep(8)
#             if len(sc)!=1:
#                 CURRENT_URL = driver.current_url
#                 driver.get(CURRENT_URL)
#                 time.sleep(5)
            size_n_color()
            
            # availability
            d = driver.find_element(by=By.CLASS_NAME, value='product-add-to-cart')
            availability_text =  d.text
            if "ACQUISTA" in availability_text:
                a_text = 'yes'
            else:
                a_text = 'no'
            # appending data-1    
            SIZE_VARIANT.append(size_list[j])
            AVAILABILITY.append(a_text)
            COLOR_VARIANT.append(colors_list[COL])
            SKU.append(sku)
            PRODUCT_NAME.append(title)

#####################################################################################################
        # appending data-2
        TITLE.append(title)
        PRICE.append(price)
        SHORT_DESCRIPTION.append(short_description)
        DESCRIPTION2.append(description2)
        SIZES.append(size)
        COLOR.append(colors_list[COL])
        if len(link_list)<4:
            for j in range(len(link_list),4):
                link_list.append("")

        for i in range(len(link_list)):
            IMAGES[i].append(link_list[i])


## Let's run the scraper function, defined above

In [ ]:
print("Total Products:",len(LINKS))

In [348]:
start = 0
end = len(LINKS)

for v in range(start , end):
    
    try:
    
        looprun(LINKS[v])
    except NoSuchElementException:
        pass
        
    print(v)
    
    
    
    


383
384
385
386
387
388
389
390
391
392
393


# Creating COLUMNS for excel file

### Run this cell once the above cell completes.

In [349]:

data1 = {
    "Product Name": PRODUCT_NAME,
    "SKU": SKU,
    "Color Variant": COLOR_VARIANT,
    "Size Variant": SIZE_VARIANT,
    "Availability": AVAILABILITY
}





data2 = {
    "Title":TITLE,
    "Price": PRICE,
    "Short Description": SHORT_DESCRIPTION,
    "Description 2": DESCRIPTION2,
    "Sizes": SIZES,
    "Colors": COLOR,
    "Image 1": IMAGES[0],
    "Image 2": IMAGES[1],
    "Image 3": IMAGES[2],
    "Image 4": IMAGES[3]
}


In [350]:
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)


## Loading the data to excel file

In [352]:
with pd.ExcelWriter('Data_2.xlsx', engine='xlsxwriter') as writer:
    df2.to_excel(writer, sheet_name='Sheet1', index=False)
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    cell_format = workbook.add_format({'text_wrap': True})
    worksheet.set_column('A:Z', cell_format=cell_format)
    
filename = "Data_1.xlsx"
df2.to_excel(filename, index=False)